## Calculo de tiempo promedio de horas pico de las sucursales 
### Desarrollado por:
- **Nombre:** Gabriel Chavez 
- **Fecha:** 2024-11-21

### Descripción
Tomaremos de referencia las transacciones de kielsa Honduras, donde pondremos de forma ordenada las cajas y los id de facturas, para poder calcular el tiempo promedio de horas pico de las sucursales.
- La hora de inicio sera la fecha de finazalizacion anterior.
- El tiempo promedio sera la diferencia entre la hora de inicio y la hora de fin de la transaccion.
- Organizar por tipo de transaccion y sacar el promedio.


In [1]:
import polars as pl
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import text

from sqlalchemy import create_engine
from urllib.parse import quote_plus


connection_string = (
    'mssql+pyodbc://'
    'Angel_chavez:{}@172.16.2.227\\DWHFARINTERDEV/BI_FARINTER?'
    'driver=ODBC+Driver+17+for+SQL+Server'.format(quote_plus('@ng3l_ch@v3z'))
)
engine = create_engine(connection_string)
query = text("""SELECT
    fc.Suc_Id,
    fc.Emp_Id,
    fc.Caja_Id,
    fc.Factura_Id,
    fc.Factura_Fecha,
    fc.Factura_FechaHora,
    fc.AnioMes_Id,
    fc.Cliente_Id,
    fc.MonederoTarj_Id,
    fc.Vendedor_Id,
    CASE 
        WHEN fc.Monedero_Id = '0' THEN 'No Identificada'
        ELSE md.Tipo_Plan 
    END AS Tipo_Plan,
    md.Edad,
    suc.TipoSucursal_Id,
    suc.TipoSucursal_Nombre,
    suc.Zona_Id,
    suc.Zona_Nombre
FROM
    [BI_Kielsa_Hecho_FacturaEncabezado] AS fc
    JOIN [BI_Kielsa_Dim_Monedero] AS md 
        ON md.Monedero_Id = fc.Monedero_Id AND md.Emp_Id = fc.Emp_Id       
    JOIN [BI_Kielsa_Dim_Sucursal] AS suc 
        ON suc.Sucursal_Id = fc.Suc_Id AND suc.Emp_Id = fc.Emp_Id
WHERE
    fc.AnioMes_Id IN (202410,202411, 202409)
    AND fc.Emp_Id = 1;""")

with engine.connect() as conn:
    df_base = pd.read_sql(query, conn).set_index(['Suc_Id', 'Caja_Id', 'Factura_Id']).reset_index()

In [18]:
df_base.head()

,Suc_Id,Caja_Id,Factura_Id,Emp_Id,Factura_Fecha,Factura_FechaHora,AnioMes_Id,Cliente_Id,MonederoTarj_Id,Vendedor_Id,Tipo_Plan,Edad,TipoSucursal_Id,TipoSucursal_Nombre,Zona_Id,Zona_Nombre
0,53,1,59518,1,2024-10-22,23:30:28,202410,10000001,0,10004372,No Identificada,33,7,Stand Alone,2,NORTE
1,53,1,59511,1,2024-10-22,22:26:52,202410,10000001,0,10004372,No Identificada,33,7,Stand Alone,2,NORTE
2,72,1,117448,1,2024-10-22,17:18:19,202410,10000001,0,10004087,No Identificada,33,7,Stand Alone,2,NORTE
3,124,1,135704,1,2024-10-22,17:53:35,202410,10000001,0,10004790,No Identificada,33,2,Centro Comercial,1,CENTRO SUR
4,261,1,22181,1,2024-10-22,12:47:31,202410,10000001,0,10003772,No Identificada,33,5,Estación de Servicio,1,CENTRO SUR


In [2]:
import numpy as np

# Copia de DataFrame base
df = df_base.copy()

tipos_transaccion = {
    'Monedero tercera edad': 'Tercera Edad',
    'Monedero Cuarta Edad': 'Cuarta Edad',
    'Monedero todo publico': 'Todo Publico',
    'Monedero Clínica TP': 'Todo Publico',
    'Monedero Clínica TE': 'Tercera Edad',
    'No Identificada': 'No Identificada',
}

# Mapear los tipos de transacciones y eliminar las no relevantes
df['Tipo_Plan'] = df['Tipo_Plan'].map(tipos_transaccion)
df = df.dropna(subset=['Tipo_Plan'])


# Convertir Factura_FechaHora a cadena para agregar fecha ficticia
df['Factura_FechaHora'] = df['Factura_FechaHora'].astype(str)
df['Factura_FechaHora'] = '1900-01-01 ' + df['Factura_FechaHora']

# Convertir Factura_FechaHora a tipo datetime
df['Factura_FechaHora'] = pd.to_datetime(df['Factura_FechaHora'], errors='coerce')

# Asegurarse de que Factura_Fecha sea tipo datetime
df['Factura_Fecha'] = pd.to_datetime(df['Factura_Fecha'])

# Ordenar el DataFrame
df = df.sort_values(by=['Suc_Id', 'Caja_Id', 'Factura_Fecha', 'Factura_FechaHora'], ascending=True)

# Agregar una columna para la hora
df['Hora'] = df['Factura_FechaHora'].dt.hour

# Crear columna Factura_Inicio_FechaHora solo si la fecha, la sucursal y la caja son las mismas
df['Factura_Inicio_FechaHora'] = np.where(
    (df['Suc_Id'] == df['Suc_Id'].shift(1)) & 
    (df['Caja_Id'] == df['Caja_Id'].shift(1)) & 
    (df['Factura_Fecha'] == df['Factura_Fecha'].shift(1)) &
    (df['Hora'] == df['Hora'].shift(1)),
    df['Factura_FechaHora'].shift(1),
    pd.NaT 
)
# Asegurarse de que la columna Factura_Inicio_FechaHora sea tipo datetime
df['Factura_Inicio_FechaHora'] = pd.to_datetime(df['Factura_Inicio_FechaHora'], errors='coerce')

# Calcular el tiempo transcurrido
df['Tiempo_Transcurrido'] = df['Factura_FechaHora'] - df['Factura_Inicio_FechaHora']

# convertir el tiempo transcurrido a minutos
df['Tiempo_Transcurrido'] = df['Tiempo_Transcurrido'].dt.total_seconds() / 60
# excluir columnas con un Factura_Inicio_FechaHora igual a NaT
df = df.dropna(subset=['Factura_Inicio_FechaHora'])

In [3]:

# Agrupar por hora y tipo (por ejemplo, Tipo_Plan) y calcular métricas
resultados = df.groupby(['Hora', 'Tipo_Plan', 'Suc_Id', 'Caja_Id', 'Factura_Fecha']).agg(
    Tiempo_Promedio=('Tiempo_Transcurrido', 'mean'),
    Cantidad_Transacciones=('Factura_Id', 'count')
).reset_index()

resultados.Tiempo_Promedio.max()

59.7

In [12]:
# Identificar horas pico basadas en el mayor número de transacciones
horas_pico_por_sucursal_dia = resultados.groupby(['Suc_Id', 'Factura_Fecha', 'Hora']).agg(
    Cantidad_Transacciones=('Cantidad_Transacciones', 'sum')
).reset_index()
horas_pico_por_sucursal_dia['Hora_Pico'] = horas_pico_por_sucursal_dia.groupby(['Suc_Id', 'Factura_Fecha'])['Cantidad_Transacciones'].transform(max) == horas_pico_por_sucursal_dia['Cantidad_Transacciones']



In [13]:
print(horas_pico_por_sucursal_dia[horas_pico_por_sucursal_dia['Hora_Pico']])

        Suc_Id Factura_Fecha  Hora  Cantidad_Transacciones  Hora_Pico
3            1    2024-09-01    11                       5       True
20           1    2024-09-02    19                       7       True
33           1    2024-09-03    20                       8       True
46           1    2024-09-04    18                      11       True
59           1    2024-09-05    19                       6       True
...        ...           ...   ...                     ...        ...
327084     328    2024-11-27    12                       4       True
327093     328    2024-11-28    13                       3       True
327098     328    2024-11-29    10                       2       True
327102     328    2024-11-29    17                       2       True
327114     328    2024-11-30    21                       4       True

[33599 rows x 5 columns]


In [14]:
# Filtrar solo las horas pico
horas_pico_solo_dia = horas_pico_por_sucursal_dia[horas_pico_por_sucursal_dia['Hora_Pico']]


In [15]:
# Calcular los promedios por tipo de transacción considerando solo las horas pico
promedios_horas_pico = resultados.merge(
    horas_pico_solo_dia[['Suc_Id', 'Factura_Fecha', 'Hora']],
    on=['Suc_Id', 'Factura_Fecha', 'Hora'],
    how='inner'
)

# Ordenar las horas pico por la cantidad de transacciones, de mayor a menor
horas_pico_ordenadas = horas_pico_solo_dia.sort_values(by='Cantidad_Transacciones', ascending=False)

# Calcular el 25% del total de las horas pico
n_top_25 = int(len(horas_pico_ordenadas) * 1)

# Seleccionar el top 25% de las mejores horas pico
top_25_horas_pico = horas_pico_ordenadas.head(n_top_25)

top_25_horas_pico_completo = resultados.merge(
    top_25_horas_pico[['Suc_Id', 'Factura_Fecha', 'Hora']],
    on=['Suc_Id', 'Factura_Fecha', 'Hora'],
    how='inner'
)

In [16]:
# identificar las cajas mas top
cajas_top_por_hora = top_25_horas_pico_completo.groupby(['Suc_Id', 'Factura_Fecha', 'Hora', 'Caja_Id']).agg(
    Cantidad_Transacciones=('Cantidad_Transacciones', 'sum')
).reset_index()

# Ordenar las cajas por la cantidad de transacciones dentro de cada hora pico
cajas_top_por_hora = cajas_top_por_hora.sort_values(by=['Suc_Id', 'Factura_Fecha', 'Hora', 'Cantidad_Transacciones'], ascending=[True, True, True, False])

# Seleccionar las cajas con más transacciones dentro de cada hora pico
cajas_top_por_hora['Caja_Top'] = cajas_top_por_hora.groupby(['Suc_Id', 'Factura_Fecha', 'Hora'])['Cantidad_Transacciones'].transform(max) == cajas_top_por_hora['Cantidad_Transacciones']

# Filtrar solo las cajas top
top_cajas_final = cajas_top_por_hora[cajas_top_por_hora['Caja_Top']]

# Unir la información de las cajas top con los resultados originales
top_25_horas_pico_cajas_top = top_25_horas_pico_completo.merge(
    top_cajas_final[['Suc_Id', 'Factura_Fecha', 'Hora', 'Caja_Id']],
    on=['Suc_Id', 'Factura_Fecha', 'Hora', 'Caja_Id'],
    how='inner'
)

# Resultado final: top 25% de horas pico con las cajas más activas
top_25_horas_pico_cajas_top

,Hora,Tipo_Plan,Suc_Id,Caja_Id,Factura_Fecha,Tiempo_Promedio,Cantidad_Transacciones
0,0,No Identificada,3,1,2024-10-04,2.940000,5
1,0,Tercera Edad,3,1,2024-10-04,6.933333,1
2,0,Todo Publico,3,1,2024-10-04,3.945833,8
3,0,No Identificada,3,2,2024-09-29,4.491667,6
4,0,Tercera Edad,3,2,2024-09-29,6.775000,2
...,...,...,...,...,...,...,...
84761,23,Todo Publico,84,1,2024-09-09,3.894444,6
84762,23,Todo Publico,84,1,2024-11-07,0.881944,12
84763,23,Todo Publico,84,1,2024-11-09,2.680303,11
84764,23,Todo Publico,84,1,2024-11-24,4.680952,7


In [20]:
# Agrupar por tipo de transacción y calcular el tiempo promedio en horas pico
promedios_horas_pico_general = top_25_horas_pico_cajas_top.groupby(['Suc_Id', 'Tipo_Plan']).agg(
    Tiempo_Promedio=('Tiempo_Promedio', 'mean')
).reset_index()

print("Promedio de tiempo en horas pico por tipo de transacción y sucursal")
print(promedios_horas_pico_general)
promedios_horas_pico_general.to_excel('promedios_horas_pico_general.xlsx', index=False)


Promedio de tiempo en horas pico por tipo de transacción y sucursal
      Suc_Id        Tipo_Plan  Tiempo_Promedio
0          1      Cuarta Edad         9.777778
1          1  No Identificada         7.039611
2          1     Tercera Edad         7.274660
3          1     Todo Publico         6.863910
4          2      Cuarta Edad        14.850000
...      ...              ...              ...
1159     327     Todo Publico         6.352619
1160     328      Cuarta Edad        11.208333
1161     328  No Identificada        12.011518
1162     328     Tercera Edad        12.151473
1163     328     Todo Publico        10.080350

[1164 rows x 3 columns]


In [18]:
promedio_general_por_tipo = top_25_horas_pico_cajas_top.groupby('Tipo_Plan').agg(
    Tiempo_Promedio_Tipo=('Tiempo_Promedio', 'mean')
).reset_index()

# Resultados
print("Promedios Generales de Tiempo en Horas Pico por Tipo de Transacción:")
print(promedio_general_por_tipo)


Promedios Generales de Tiempo en Horas Pico por Tipo de Transacción:
         Tipo_Plan  Tiempo_Promedio_Tipo
0      Cuarta Edad              7.576983
1  No Identificada              5.650787
2     Tercera Edad              7.319969
3     Todo Publico              6.775179


In [19]:
# Resultados
promedios_horas_pico_por_sucursal = top_25_horas_pico_cajas_top.groupby('Suc_Id').agg(
    Tiempo_Promedio_Sucursal=('Tiempo_Promedio', 'mean')
).reset_index()

# Resultados
print("Promedios de Tiempo en Horas Pico por Sucursal:")
print(promedios_horas_pico_por_sucursal)




Promedios de Tiempo en Horas Pico por Sucursal:
     Suc_Id  Tiempo_Promedio_Sucursal
0         1                  7.068693
1         2                  8.005666
2         3                  5.675338
3         4                  6.435392
4         5                  5.708037
..      ...                       ...
296     324                  9.609773
297     325                 11.031705
298     326                  8.508418
299     327                  6.096008
300     328                 11.539204

[301 rows x 2 columns]
